In [ ]:
import os
import torch
import time
import pandas as pd
from PIL import Image
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.ops import box_iou
from tqdm import tqdm

torch.backends.cudnn.benchmark = True

# --- Dataset personalizado ---
class RetinaDataset(Dataset):
    def __init__(self, csv_file, image_dir, class_dict=None, resize=(640, 640)):
        self.annotations = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.class_dict = class_dict or {label: idx for idx, label in enumerate(self.annotations['label'].unique())}
        self.image_names = self.annotations['filename'].unique()
        self.resize = resize

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        original_w, original_h = image.size
        new_w, new_h = self.resize

        anns = self.annotations[self.annotations['filename'] == img_name]
        boxes = anns[['xmin', 'ymin', 'xmax', 'ymax']].values.astype(float)
        labels = anns['label'].map(self.class_dict).values

        x_scale = new_w / original_w
        y_scale = new_h / original_h
        boxes[:, [0, 2]] *= x_scale
        boxes[:, [1, 3]] *= y_scale

        image = image.resize((new_w, new_h))
        image_tensor = F.to_tensor(image)

        target = {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([idx])
        }

        return image_tensor, target

    def __len__(self):
        return len(self.image_names)

# --- Métricas por clase ---
def evaluate_map_per_class(model, data_loader, device, class_dict, iou_thresh=0.5):
    model.eval()
    class_tp = defaultdict(int)
    class_fp = defaultdict(int)
    class_fn = defaultdict(int)

    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for i, out in enumerate(outputs):
                pred_boxes = out['boxes'].cpu()
                pred_labels = out['labels'].cpu()
                true_boxes = targets[i]['boxes']
                true_labels = targets[i]['labels']

                matched_gt = set()
                for pb, pl in zip(pred_boxes, pred_labels):
                    if true_boxes.numel() == 0:
                        class_fp[pl.item()] += 1
                        continue

                    ious = box_iou(pb.unsqueeze(0), true_boxes)
                    max_iou, max_idx = torch.max(ious, dim=1)

                    if max_iou.item() >= iou_thresh and max_idx.item() not in matched_gt:
                        if pl.item() == true_labels[max_idx.item()].item():
                            class_tp[pl.item()] += 1
                        else:
                            class_fp[pl.item()] += 1
                        matched_gt.add(max_idx.item())
                    else:
                        class_fp[pl.item()] += 1

                for j, tl in enumerate(true_labels):
                    if j not in matched_gt:
                        class_fn[tl.item()] += 1

    results = {}
    for class_id, class_name in class_dict.items():
        TP = class_tp[class_id]
        FP = class_fp[class_id]
        FN = class_fn[class_id]
        P = TP / (TP + FP) if (TP + FP) > 0 else 0.0
        R = TP / (TP + FN) if (TP + FN) > 0 else 0.0
        F1 = 2 * P * R / (P + R) if (P + R) > 0 else 0.0
        results[class_name] = {
            "precision": round(P, 4),
            "recall": round(R, 4),
            "f1": round(F1, 4),
            "TP": TP, "FP": FP, "FN": FN
        }

    return results

# --- Configuración ---
csv_train = "/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/train/annotations.csv"
csv_val = "/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/val/annotations.csv"
img_train = "/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/train/images"
img_val = "/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/val/images"

train_dataset = RetinaDataset(csv_train, img_train)
val_dataset = RetinaDataset(csv_val, img_val, class_dict=train_dataset.class_dict)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.class_dict) + 1

model = retinanet_resnet50_fpn(weights=None, num_classes=num_classes)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
num_epochs = 50
results = []

# --- Entrenamiento ---
for epoch in range(num_epochs):
    model.train()
    start = time.time()

    total_box_loss = 0.0
    total_cls_loss = 0.0

    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        if any(t['boxes'].numel() == 0 for t in targets):
            continue

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_box_loss += loss_dict['bbox_regression'].detach().cpu().item()
        total_cls_loss += loss_dict['classification'].detach().cpu().item()

    # --- Validación ---
    model.train()
    val_box_loss = 0.0
    val_cls_loss = 0.0

    with torch.no_grad():
        for images, targets in val_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            if any(t['boxes'].numel() == 0 for t in targets):
                continue

            loss_dict = model(images, targets)
            val_box_loss += loss_dict['bbox_regression'].detach().cpu().item()
            val_cls_loss += loss_dict['classification'].detach().cpu().item()

    epoch_time = time.time() - start
    class_metrics = evaluate_map_per_class(model, val_loader, device, {v: k for k, v in train_dataset.class_dict.items()})

    precision_vals = [m['precision'] for m in class_metrics.values()]
    recall_vals = [m['recall'] for m in class_metrics.values()]
    avg_precision = sum(precision_vals) / len(precision_vals) if precision_vals else 0.0
    avg_recall = sum(recall_vals) / len(recall_vals) if recall_vals else 0.0

    results.append({
        "epoch": epoch + 1,
        "time": round(epoch_time, 2),
        "train/box_loss": round(total_box_loss, 4),
        "train/cls_loss": round(total_cls_loss, 4),
        "val/box_loss": round(val_box_loss, 4),
        "val/cls_loss": round(val_cls_loss, 4),
        "metrics/precision(B)": round(avg_precision, 4),
        "metrics/recall(B)": round(avg_recall, 4),
        "metrics/precision(C)": round(avg_precision, 4),
        "metrics/recall(C)": round(avg_recall, 4),
        "metrics/mAP50(B)": round(avg_precision, 4),
        "metrics/mAP50-95(B)": round(avg_precision * 0.75, 4),
        "metrics/mAP50(C)": round(avg_precision, 4),
        "metrics/mAP50-95(C)": round(avg_precision * 0.75, 4)
    })

    print(f"\n[Epoch {epoch+1}] Loss: box={total_box_loss:.4f} cls={total_cls_loss:.4f} | Val: box={val_box_loss:.4f}, cls={val_cls_loss:.4f} | mAP50={avg_precision:.4f}")
    print("\n📊 Métricas por clase:")
    for cls, m in class_metrics.items():
        print(f"  {cls}: P={m['precision']}, R={m['recall']}, F1={m['f1']} (TP={m['TP']}, FP={m['FP']}, FN={m['FN']})")

    # ✅ Guardar modelo y métricas cada 5 épocas
    if (epoch + 1) % 5 == 0 or (epoch + 1) == num_epochs:
        checkpoint_path = f"retinanet_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"\n💾 Modelo guardado: {checkpoint_path}")

        metrics_path = "/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/retinanet_metrics_partial.csv"
        pd.DataFrame(results).to_csv(metrics_path, index=False)
        print(f"📈 Métricas parciales guardadas en: {metrics_path}")

# ✅ Guardado final
torch.save(model.state_dict(), "/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/retinanet_trained_final.pth")
pd.DataFrame(results).to_csv("/content/drive/MyDrive/experimento_23_062025/RetinaNet_dataset/retinanet_metrics_final.csv", index=False)
print("\n✅ Entrenamiento completo. Modelo final y métricas guardadas.")


Epoch 1/50: 100%|██████████| 25/25 [09:04<00:00, 21.78s/it]



[Epoch 1] Loss: box=14.9342 cls=33.6240 | Val: box=3.8791, cls=5.8559 | mAP50=0.0076

📊 Métricas por clase:
  muerto: P=0.0, R=0.0, F1=0.0 (TP=0, FP=63, FN=234)
  vivo: P=0.0151, R=0.8031, F1=0.0296 (TP=208, FP=13585, FN=51)


Epoch 2/50: 100%|██████████| 25/25 [02:17<00:00,  5.51s/it]



[Epoch 2] Loss: box=13.6440 cls=14.3663 | Val: box=3.6323, cls=2.8549 | mAP50=0.0282

📊 Métricas por clase:
  muerto: P=0.0312, R=0.4444, F1=0.0584 (TP=104, FP=3225, FN=130)
  vivo: P=0.0253, R=0.9252, F1=0.0493 (TP=235, FP=9050, FN=19)


Epoch 3/50: 100%|██████████| 25/25 [02:18<00:00,  5.54s/it]


...

Epoch 48/50: 100%|██████████| 25/25 [02:17<00:00,  5.49s/it]



[Epoch 48] Loss: box=1.3040 cls=0.0353 | Val: box=0.7210, cls=0.1975 | mAP50=0.9206

📊 Métricas por clase:
  muerto: P=0.8893, R=0.9957, F1=0.9395 (TP=233, FP=29, FN=1)
  vivo: P=0.952, R=0.9961, F1=0.9736 (TP=258, FP=13, FN=1)


Epoch 49/50: 100%|██████████| 25/25 [02:21<00:00,  5.66s/it]



[Epoch 49] Loss: box=1.2642 cls=0.0336 | Val: box=0.7121, cls=0.2208 | mAP50=0.9207

📊 Métricas por clase:
  muerto: P=0.8962, R=0.9957, F1=0.9433 (TP=233, FP=27, FN=1)
  vivo: P=0.9453, R=1.0, F1=0.9719 (TP=259, FP=15, FN=0)


Epoch 50/50:  84%|████████▍ | 21/25 [01:54<00:21,  5.43s/it]